In [1]:
# Get contends from an article and get each sentences
# Read sentences based on "，", not " " or "\n"
# Break sentences in words
# Then remove stop words
# Pool all this sentences in a list
# Save list in a csv file

import re
import jieba
import zhon


# Receive a list containing lines of an article
# Remove unwanted characters
# Retrun a list containing sentences of an article
def sentence_check(a):
    # The faster way is creating a list, then checking any unwanted characters before replacing it. 
    # However, this way produces multiple output, which increase some complexity 
    print("run sentence check")
    check_list=a
    return_list=[]
    pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）' 
    
    for element in check_list:
        element = re.split(pattern,element)
        for t in element:
            if t != '':
                return_list.append(t)
    
    # Remove \n in list
    return_list = [x for x in return_list if x != '\n']
    return(return_list)


# Read sentences of an article and process them as a bag of words
# Use jieba module to recognize chinese words
# Need a dictionary of common tradition Chinese words
# Need a dictionary of stop words
# Return a list of lists of words (still keep the sentence information)
def word(a):
    print("Run word, powered by JieBa")
    input_list = a
    jieba.set_dictionary('dict.txt')
    stops = open('stop.txt', 'r', encoding='utf8') 
    stopword = stops.read().split('\n')
    stops.close()
    
    # Update stop words
    stopword.append('\n')
    
    new_list=[]
    i = 0
    
    # Remove any stop words from list
    while i < len(input_list):
        element = input_list[i]
        new_list.append([t for t in jieba.cut(element) if t not in stopword])
        i+=1
    
    return(new_list)
    
def main():
    print("Run main")
    fp = open('test.txt','r')
    lines = fp.readlines()
    fp.close()
    sentence_list = sentence_check(lines)
    word_list=word(sentence_list)
    print(word_list) 
    
if __name__ == "__main__":
    print("initialize")
    main() 

Building prefix dict from /Users/hsieh/Documents/ptt/dict.txt ...
Loading model from cache /var/folders/54/w85qh3vd4xv5v6qz8w4t70940000gn/T/jieba.ubaa545e936adb4bd7ef290058a1bf422.cache


initialize
Run main
run sentence check
Run word, powered by JieBa


Loading model cost 0.728 seconds.
Prefix dict has been built succesfully.


[['某天', '一群', '女孩', '朋友', '下午茶', '後'], ['聊到', '一起', '出國', '話題'], ['當天', '下午', '意外', '決定', '中秋', '連假', '一趟', '沖繩'], ['這趟', '旅行', '一致', '想法'], ['游泳'], ['潛水'], ['做', '水上', '運動'], ['悠遊', '漫步', '沖繩', '行'], ['海邊', '喝', '咖啡'], ['至少', '一晚', '住', '看到', '海邊', '飯店'], ['團員', '人', '好', '請假'], ['目標', '四天', '三夜'], [], ['收看', '這篇', '文章', '想法', '天數'], ['第一次', '沖繩'], ['這篇', '適合', '參考'], ['航空', '選擇'], ['台灣', '沖繩', '飛機', '時間', '一個', '小時'], ['傳統', '航空', '華航', '長榮', '之外'], ['廉價', '航空', '虎航'], ['樂桃'], ['香草'], ['這次', '一起', '玩', '女孩兒', '總共', '人'], ['搶票', '麻煩', '四天', '時間'], ['考慮', '廉價', '航空'], ['一番', '比價', '之後'], ['透過', '雄獅', '旅行社', '買', '長榮', '來回', '機票', '一晚', '國際'], ['通', '住宿'], ['一個', '人', '費用', '一萬二', '台幣'], ['買到', '時間', '好'], ['早上', '06'], ['45', '出發'], ['抵達', '沖繩', '那霸', '約', '09'], ['15'], ['回程', '時間'], ['晚上', '20'], ['20', '起飛'], ['抵達', '台灣', '約', '20'], ['55'], ['等於', '玩', '四個', '整天'], ['香草', '航空', '時間', '買到', '長榮航'], ['空', '早', '晚回'], ['飛行', '時間', '短', '覺得', '搭', '廉價', '航空', '很棒', '選擇'], ['這次', '團員', 

In [92]:
import sqlite3
import requests
import csv
from bs4 import BeautifulSoup


# Enter the subforum which you want to scrape data from
# Subforum name is case sensitive

conn = sqlite3.connect('ptt.db')
c = conn.cursor()
cursor = c.execute("SELECT id from test")
url_list=[]
for row in cursor:
    url_list.append(row[0])
cursor.close()
name = input("Enter which sub forum you want to scrape data from: ")
print("get data from {a}".format(a = name))

# Send http request to server

text_list=[]
for element in url_list:
    url = "https://www.ptt.cc/bbs/{name}/".format(name=name)
    url= url+element
    print(url)
    response = requests.get(url, cookies={'over18': '1'}) 
    result = response.text
    soup = BeautifulSoup(result, 'html.parser')
    soup = soup.find(id="main-content")
    for span in soup('span'):
        span.decompose()
    for div in soup('div'):
        div.decompose()
    for a in soup('a'):
        a.decompose()
    text_list.append(soup.text)
print(text_list)

Enter which sub forum you want to scrape data from: Japan_Travel
get data from Japan_Travel
https://www.ptt.cc/bbs/Japan_Travel/M.1552791579.A.86D.html
https://www.ptt.cc/bbs/Japan_Travel/M.1552742037.A.E2D.html
['\n\n2月時自己開一團帶同事去北九州，\n由於過程中報名人數不斷增加，\n最後變成14大2小的團。\n由於是第一次帶團，先前PO文詢問時，\n很多網友推文說：一定會吵架～ 記得回報啊～\n\n首先想說，我們沒吵架、真的完全沒吵架\n\n是的...只要大家都心胸開闊，哪有什麼好吵的呢...\n但看到前陣子別人分享的孝親之旅，\n如果是裏頭的"父母"，那還真的沒把握了 XDD\n所以只是運氣，真該慶幸同事都是好相處的人。\n\n------------------前言結束----------------------\n不囉嗦遊記，\n直接重點整理好了。\n\n\n\n\n\n\n\n\n\n2/20(三) 虎航0645-1000福岡 -> 機場巴士到博多站換北九州三日鐵路周遊券+劃位(此時介紹其他人去吃半熟咖哩蛋麵包、可頌，避免無聊) -> 搭地鐵前往中洲川端(介紹離飯店最近的唐吉軻德及一蘭本店位址) -> 西鐵INN 福岡 放行李 -> 帶大家步行到天神，午餐吃純情拉麵本店(大家同意排隊就吃囉，有獲好評) -> 自由行動亂逛(我把讓大家認識環境作為優先) -> 部分人晚餐 博多 祇園 鉄なべ(我請客但soso，我在日本從沒吃過滿意的煎餃) -> 回程唐吉軻德買酒續攤\n\n2/21(四) 經飯店對面的Acros福岡地下通道前往天神(我每次帶路都會走不同路線，希望讓大家盡快熟悉環境) -> 讓大家排隊購買福岡City PASS太宰府版(感覺得出大家對買票券這種行為感到新奇) -> 前往太宰府(自由行動，包括回程都各自依需求) -> Outlet組 姪浜駅會合搭昭和巴士前往瑪莉諾雅(先跟大家約定晚上7點30分會合一起去吃燒肉吃到飽) -> 因每人逛Outlet時間不同，各自回到飯店(其中一組母女沒想買什麼，所以我只好先早早結束帶她們回去) -> 前往

In [91]:
soup = BeautifulSoup(result, 'html.parser')
soup = soup.find(id="main-content")
for span in soup('span'):
    span.decompose()
for div in soup('div'):
    div.decompose()
for a in soup('a'):
    a.decompose()

soup.text

'\n今天(3/16)起，上越新幹線將導入E7系的車輛，今天一大早在新潟車站還有辦始發車的出\n發典禮哦\n\n所謂出發典禮就是東日本JR的高官舉著旗子喊「出發!」的活動 (我猜的)\n\n其實他們喊了什麼我是不知道，因為我就在那台車子裡頭(噗)\n\n\n\n今天起配合新的新幹線，時刻表也更換了，不用說，3/16對上越新幹線是個大日子\n\n車站內的海報，甚至綠色窗口的售票機…E7 E7 E7…\n\n目前上越新幹線有E2系和E4系兩種車型，其中雙層的E4系因為太過老舊，將在2020年底全\n部汰換成E7系\n\n而且因為換了E7系，上越新幹線終於有了最頂級的GRAN車廂了!\n\n昨天(3/15)晚上抱著”反正GRAN一定賣光了”的心態，上網看了一下，竟然剛好剩下2個!\n\n於是立馬得標，並在新潟車站摩斯都還沒開的時候就抵達車站，隨手買個車站內的三明治\n和麵包，還不錯吃\n\n在E7進站月台的對面是現美新幹線，以前有分享過，好像聽說內部設計有些更新，\n有機會再試試看\n\n我們搭乘7點49分往東京方向列車，大大有看到TOKI 7:49 TOKYO字樣旁邊有個6角形的標\n誌嗎? 它就是…\n\nGran Class的標誌\n\n可能對有些大大來說還不太清楚Gran Class，簡單解釋一下\n\n不只是新幹線，連部分特急列車都有綠色車廂對吧!\n\n綠色車廂就如這種感覺(E7內部實際樣式)\n\n比起一般座位(說真的E7的一般座位也是很好坐拉! 而且有插頭的TOKI少少) 整個座椅、\n車廂質感會再好一點\n\n那Gran Class呢… 如果用飛機來比喻\n\n約為經濟艙(一般)-豪華經濟艙(綠色車廂)-商務艙(Gran)的感覺吧\n\n(換上拖鞋搭乘更舒適)\n\n除了Gran車廂之外，最後想請大家看，這台上越新幹線\n\n有別於以往的TOKI，有加上象徵TOKI的粉紅色\n\n以及新的車體外有畫上代表新潟的稻穗(INAHO)和朱鷺的羽毛，這個設計圖樣是只有一年\n的期間限定喔!\n\n今天這台E7通過各大站時，都受到不少注目的樣子，尤其是車站大大的眼睛\n\n看著綠色車廂和Gran車廂的時間好像比較長，是我的錯覺嗎?XD\n\n--\n\n\nIG同好召募中\n\n\n相簿\n\n--\n'